Write some markdown to explain that this notebook will be mainly used for the capstone project.
Import the pandas library as pd.
Import the Numpy library as np.
Print the following the statement: Hello Capstone Project Course!

In [ ]:
#url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#from bs4 import BeautifulSoup
#import requests

#r=requests.get(url)
#content=r.content
#soup=BeautifulSoup(content,"html.parser")

In [ ]:
#print(soup)

In [ ]:
#wikitable=soup.find('table',class_='wikitable sortable')
#print(wikitable)
#now the information is stored in wikitable

In [ ]:
#from pandas.io.json import json_normalize
#df = json_normalize(wikitable)

# Step1: read the postal table from an url link (see below)
1. define a URL
2. read the html table into data frame
3. remove the rows of Borough="Not assigned" 
4. reset the index
5. change the value in Neighborhood="Not assigned" to the values of Borough

[The above code does not change anything as in this table, when the "Borough"="Not assigned" is removed, there is no "Neighborhood"="Not assigned"]
[But the code is made and tested to make sure the function is working]
******************************************************************************************************************************************************

In [179]:
!pip install lxml
import pandas as pd # library for data analsysis

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)[0]
#dfs is a dataframe object

In [180]:
dfs.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [181]:
df=dfs[dfs.Borough!="Not assigned"]
df=df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [182]:
for i in range(103):
    value=df.iloc[i]["Neighbourhood"]
    if value=="Not assigned":
        df.iloc[i]["Neighbourhood"]=df.iloc[i]["Borough"]
        print("find a NOT ASSIGNED")
    
df.head(5)
#The above code is not actually used as there is no "Not assigned" value in column "Beighbourhood"

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [183]:
df.shape

(103, 3)

# End of Step1: read the postal table from an url link

# Step 2: read the geo data from an url link and merge it with the table in setp1(see below)
1. define a URL
2. use pandas.read_csv to read the geo data into a data frame
3. merge this data frame with the postal code-borough-neighborhood dataframe

In [184]:
url="http://cocl.us/Geospatial_data"

import pandas
geo_df = pandas.read_csv(url)

merge_df=(df.join(geo_df.set_index('Postal Code'),on='Postal Code')).copy()

#manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merge_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [185]:
merge_df["Select"]=False
#Add an empty column "Select" to merge_df
merge_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Select
0,M3A,North York,Parkwoods,43.753259,-79.329656,False
1,M4A,North York,Victoria Village,43.725882,-79.315572,False
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,False
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,False
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,False


# End of Step 2: read the geo data from an url link and merge it with the table in setp1

# Step 3: make the GET oepration to retrive the venue info, make clustering and plot on the map (see below)
1. Make query to get the niformation of venues
2. Use the K-means to do the clustering (here I defined only 3 clusters)
3. Plot the vanues on the map with different colors presenting clusters

In [186]:
rows=merge_df.shape[0]


for i in range(rows):
    string=merge_df.iloc[i]["Borough"]
    if string.find('Toronto')==-1:
        merge_df.iloc[i,5]=False
    else:
        merge_df.iloc[i,5]=True

merge_df.head(10)
# Now all the rows with Borough has word "Toronto" have been marked "True" in column "Select"
# Now merge_df is 103x6

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Select
0,M3A,North York,Parkwoods,43.753259,-79.329656,False
1,M4A,North York,Victoria Village,43.725882,-79.315572,False
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,True
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,False
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,True
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,False
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,False
7,M3B,North York,Don Mills,43.745906,-79.352188,False
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,False
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,True


In [187]:
merge_df.shape

(103, 6)

In [188]:
toronto_df=merge_df[merge_df.Select==True]
toronto_df.reset_index(drop=True)
toronto_df.shape

# Now we have a DataFrame with dimension of 39x6. Menaing 39 neighbourhoods have belong to Borough with "Toronto" word in their name

number_of_neighbourhood=toronto_df.shape[0]
number_of_neighbourhood

39

In [189]:
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Select
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,True
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,True
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,True
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,True
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,True


In [190]:
! pip install folium==0.5.0
import folium

In [191]:
latitude=43.6548
longitude=-79.3883
map_toronto=folium.Map(location=[latitude,longitude], zoom_start=12)

for lat, lng, borough, postal_code in zip(toronto_df["Latitude"],toronto_df["Longitude"],toronto_df["Borough"],toronto_df["Postal Code"]):
    label='{},{}'.format(postal_code,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto    


In [192]:
toronto_df.head(3)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Select
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,True
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,True
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,True


In [193]:
import requests # library to handle requests
from pandas.io.json import json_normalize

In [194]:
CLIENT_ID = '4BLKWPUP0EYC2U4UGAI3YQVWR5NDYHVDF1VNHK41DHG1EWVV' # your Foursquare ID
CLIENT_SECRET = '4HYGXEIO145QTOAN4BOHOQNFWDWUSKF3HI3Q2EWZ4GHJIG3J' # your Foursquare Secret
ACCESS_TOKEN = 'XECU25SP52OQUWQJK2DPP42QKAZDRPTHDWOPZCKPJ1DQLGLK' # your FourSquare Access Token
VERSION = '20180605'

neibourghhood_latitude=toronto_df.iloc[0,3]
neibourghhood_longitude=toronto_df.iloc[0,4]
neibourghhood_name=toronto_df.iloc[0,2]
search_query = '' 
radius = 500

latitude=neibourghhood_latitude
longitude=neibourghhood_longitude
LIMIT = 30

In [195]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

#Make a query with the information of first neighbourhood

In [196]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
#Define a function to get the category type info

In [197]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues)

#Retrive the venues list from result
#And convert it to a dataframe called "nearby_venues". it is a 30x8 DF

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


In [198]:
nearby_venues.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,venuePage.id
0,5bdc6c2bba57b4002c4c71a8,Oldtown Bodega,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1610831148,False,43.653966,-79.360752,"[{'label': 'display', 'lat': 43.653966, 'lng':...",34,M5A 1L6,CA,Toronto,ON,Canada,[Toronto ON M5A 1L6],NaN,NaN,NaN
1,4bc70f5d14d7952126a066e9,Sackville Playground,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1610831148,False,43.654656,-79.359871,"[{'label': 'display', 'lat': 43.65465604258614...",75,NaN,CA,Toronto,ON,Canada,"[420 king st E, Toronto ON]",420 king st E,NaN,NaN
2,50760559e4b0e8c7babe2497,Body Blitz Spa East,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1610831148,False,43.654735,-79.359874,"[{'label': 'display', 'lat': 43.65473505045365...",80,M5A 1L9,CA,Toronto,ON,Canada,[497 King Street East (btwn Sackville St and S...,497 King Street East,btwn Sackville St and Sumach St,NaN
3,53b8466a498e83df908c3f21,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1610831148,False,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",122,NaN,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON]",368 King St E,at Trinity St,NaN
4,4dc9d4d9d16495ca5add0803,Cam's Auto Service,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",v-1610831148,False,43.654195,-79.360545,"[{'label': 'display', 'lat': 43.65419500779484...",10,M4M 2T7,CA,Toronto,ON,Canada,"[475 King Street East (Sackville Street), Toro...",475 King Street East,Sackville Street,NaN


In [199]:
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
filtered_columns
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

#Cut out only the columns needed, add a column "categories" with the clean categories information.
#nearby_venues is now a 30x4 DF
nearby_venues.head()

,name,categories,location.lat,location.lng
0,Oldtown Bodega,Café,43.653966,-79.360752
1,Sackville Playground,Park,43.654656,-79.359871
2,Body Blitz Spa East,Spa,43.654735,-79.359874
3,Tandem Coffee,Coffee Shop,43.653559,-79.361809
4,Cam's Auto Service,Automotive Shop,43.654195,-79.360545


In [200]:
nearby_venues.shape

(30, 4)

In [201]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
#Define a function to search for venues 
#Input: names, latitudes, longitudes, radius
#OutPut: a list of venues with columns "Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category

In [202]:
toronto_df.head(2)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Select
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,True
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,True


In [203]:
names=toronto_df['Neighbourhood']
latitudes=toronto_df['Latitude']
longitudes=toronto_df['Longitude']
toronto_venues=getNearbyVenues(names, latitudes, longitudes, radius=500)
# use the function defined to get the nearby venue information
# toronto_venues is a 855x7 DF wit columns "Neighborhood" , "Neighborhood Latitude", "Neighborhood Longitude", "Venue", "Venue Latitude", "Venue Longitude", "Venue Category"

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [204]:
toronto_venues.groupby('Neighborhood').count()
# Be careful with the spelling of "Neighbourhood". In the result of query, it is spelled as 'Neighborhood'

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [205]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
#Now we know there are 196 unique categories of those venues

There are 196 uniques categories.


In [206]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot is now a 855x196 DF

In [207]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#toronto_grouped is a 39 rows × 196 columns DF

In [208]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
#Print out the top 5 venues in the Toronto neighbourhoods

----Berczy Park----
                venue  freq
0            Beer Bar  0.07
1      Farmers Market  0.07
2         Coffee Shop  0.07
3        Cocktail Bar  0.07
4  Seafood Restaurant  0.07


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.13
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3                  Bakery  0.09
4  Furniture / Home Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0         Yoga Studio  0.06
1                 Spa  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.21
1    Airport Lounge  0.14
2  Airport Terminal  0.14
3   Harbor / Marina  0.07
4           Airport  0.07


----Central Bay Street----


In [209]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
#Define a function to calculate the popular of vanues

In [210]:
import numpy as np

num_top_venues = 6
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

#neighborhoods_venues_sorted is now a 39x7 DF, the column 1 to 6 are the sorted categories of a venue

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Berczy Park,Farmers Market,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Nightclub,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Pizza Place,Light Rail Station,Skate Park,Burrito Place,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Coffee Shop,Rental Car Location
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Poke Place,Discount Store,Sandwich Place


In [211]:
from sklearn.cluster import KMeans 

kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:6] 

################################
# manhattan_merged->toronto_merged
# manhattan_data->df
################################
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [212]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [213]:
toronto_df.shape

(39, 6)

In [214]:
toronto_merged = toronto_df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters


In [215]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Select,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
2,Downtown Toronto,True,0,Coffee Shop,Bakery,Park,Theater,Breakfast Spot,Restaurant
4,Downtown Toronto,True,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Smoothie Shop,Sandwich Place,Burrito Place
9,Downtown Toronto,True,0,Café,Theater,Coffee Shop,College Rec Center,Plaza,Sporting Goods Shop
15,Downtown Toronto,True,0,Gastropub,Café,Farmers Market,Coffee Shop,Gym,Food Truck
24,Downtown Toronto,True,0,Coffee Shop,Café,Italian Restaurant,Poke Place,Discount Store,Sandwich Place
25,Downtown Toronto,True,0,Grocery Store,Café,Park,Athletics & Sports,Baby Store,Nightclub
30,Downtown Toronto,True,0,Coffee Shop,Café,Pizza Place,Gym / Fitness Center,Opera House,Concert Hall
31,West Toronto,True,0,Pharmacy,Bakery,Music Venue,Bank,Bar,Supermarket
37,West Toronto,True,0,Bar,Vietnamese Restaurant,Asian Restaurant,Coffee Shop,Wine Bar,New American Restaurant
42,Downtown Toronto,True,0,Café,Coffee Shop,Restaurant,Japanese Restaurant,Wine Bar,Steakhouse


In [216]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Select,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
62,Central Toronto,True,1,Garden,Home Service,Wine Bar,Dance Studio,Eastern European Restaurant,Donut Shop


In [217]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Select,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
19,East Toronto,True,2,Trail,Health Food Store,Pub,Wine Bar,Dance Studio,Eastern European Restaurant
20,Downtown Toronto,True,2,Farmers Market,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Jazz Club
36,Downtown Toronto,True,2,Plaza,Hotel,Café,Park,Deli / Bodega,Sporting Goods Shop
41,East Toronto,True,2,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Dessert Shop,Indian Restaurant
47,East Toronto,True,2,Sandwich Place,Park,Fast Food Restaurant,Ice Cream Shop,Liquor Store,Steakhouse
61,Central Toronto,True,2,Park,Swim School,Bus Line,Business Service,Electronics Store,Eastern European Restaurant
67,Central Toronto,True,2,Hotel,Park,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Department Store
68,Central Toronto,True,2,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Wine Bar,Deli / Bodega
75,West Toronto,True,2,Breakfast Spot,Gift Shop,Coffee Shop,Cuban Restaurant,Eastern European Restaurant,Dog Run
83,Central Toronto,True,2,Lawyer,Restaurant,Tennis Court,Wine Bar,Dance Studio,Donut Shop


# End of Step 3: make the GET oepration to retrive the venue info, make clustering and plot on the map